In [11]:
import pandas as pd
import numpy as np
import tensorflow_hub as hub
from scipy.spatial.distance import cdist, cosine, euclidean

In [2]:
embs = np.load('../react-app/api/data/reddit/funny/embs.npy')

In [5]:
use_model = hub.load('/Users/harinisuresh/Dev/tf_hub/universal-sentence-encoder_4/')

In [9]:
comments = pd.read_csv('../react-app/api/data/reddit/funny/projection_df').comment

In [103]:
def print_comments(arr):
    for (i,c) in enumerate(arr): 
        print(str(i) + ". " + c)   

In [153]:
def print_comment_diff(first, new):
    print("COMMENTS THAT WERE REMOVED: ")
    removed_arr = [c for c in first if c not in new]
    print_comments(removed_arr)
    print("\n")
    print("COMMENTS THAT WERE ADDED:")
    added_arr = [c for c in new if c not in first]
    print_comments(added_arr)
    

In [6]:
# vec: query vector (list of floats)
# all_vecs: pool of all vectors from which neighbors are retrieved (list of list of floats)
# comments: comments corresponding to the vectors in all_vecs (list of strings)
# n: number of neighbors to return (int)
# return_idx: if true, returns indices of nearest neighbors instead of the actual comments (bool)
def getKNNFromVector(vec, all_vecs, comments, n=30, return_idx=False):
    dist_vec = cdist(vec, all_vecs, 'cosine')
    top_vec_idx = np.argsort(dist_vec[0])[1:n+1]
    if return_idx: return top_vec_idx
    top_comments = np.array(comments)[top_vec_idx]
    top_comments = [c for c in top_comments]
    return top_comments

In [159]:
# anti-men comments
x = ['i hate men', 'I hate him', 'Let me guess -- you are a "Men\'s Rights" sexist Drumpf supporter? Heh.']

# general hate comments (we don't want these)
y = ['I hate you.  Well-played.',  "I don't like black people.", 'Goddamn I hate kids.', 
     'Oh shut up. You just hate women.',  'I really hate women. Off topic but nonetheless true.', 
     'I just hate groups of people in general.', "Man do I love avoiding women at all costs. This picture reminded me of how good life can get when you aren't married to feminist garbage."]

# anti-woman comments
x = ['i hate women', 'i hate her']

# more anti-woman comments
z = ['i hate women', 'i hate her', 'Ugly stupid bitch.', 'women are stupid', 'I really hate women. Off topic but nonetheless true.']

# general hate comments (we don't want these)
y = ['i hate you', 'i hate raisins', 'i hate kids']

vec = np.array(use_model(x))
vec = np.mean(vec, axis=0)

vec2 = np.array(use_model(y))
vec2 = np.mean(vec2, axis=0)

vec3 = np.array(use_model(z))
vec3 = np.mean(vec3, axis=0)

In [160]:
dist_vec = cdist([vec], embs, 'cosine')
dist_vec_2 = cdist([vec2], embs, 'cosine')
dist_vec_3 = cdist([vec3], embs, 'cosine')

## Just taking positive comments

In [161]:
similar_comments_original = getKNNFromVector([vec], embs, comments)
print_comments(similar_comments_original)

0. can't stand FIFO's
1. Oh shut up. You just hate women.
2. I hate you.
3. I really hate women. Off topic but nonetheless true.
4. I hate you.  Well-played.
5. it was 'I hate everybody'

http://www.cracked.com/quick-fixes/7-hilarious-ways-badass-movie-lines-got-ruined-by-tv-censors/
6. Goddamn I hate kids.
7. I'm not that miserable.
8. I hate raisins
9. PLEASE KILL HER
10. I'd fuck her.
11. I dnt knw u.. But i hate you
12. Ugly stupid bitch.
13. No one can hate like a god lover
14. That bitch is ugly as fuck
15. ... I'm a girl?
16. Just awful
17. That bitch be cray
18. I don't like your kind.
19. I fuckin hate that kid
20. I just hate groups of people in general.
21. STUPID LITTLE FAGMAN FUCK YOU I HATE YOU BITCH
22. That's a huuuugggee bitch!
23. What a disgusting fat whore.
24. i dont give a shit
25. I don't like black people.
26. Houston.

Haters gonna hate.
27. Fuck off

https://www.reddit.com/r/pics/comments/5k33w2/optical_illusion/
https://www.reddit.com/r/funny/comments/5jfvbj/

## Incorporating negative comments into distance metric

In [162]:
lam = 0.3

In [163]:
diff = dist_vec[0] - lam*dist_vec_2[0]

In [164]:
similar_comments_diff = np.array(comments)[np.argsort(diff)[1:31]]
print_comments(similar_comments_diff)

0. I really hate women. Off topic but nonetheless true.
1. Oh shut up. You just hate women.
2. can't stand FIFO's
3. PLEASE KILL HER
4. I'd fuck her.
5. it was 'I hate everybody'

http://www.cracked.com/quick-fixes/7-hilarious-ways-badass-movie-lines-got-ruined-by-tv-censors/
6. I hate you.
7. ... I'm a girl?
8. take that back! [women are funny](http://i.imgur.com/UnJvgpX.gifv)
XD
9. That bitch is ugly as fuck
10. I don't know her.
11. She is so hot to me.
12. LET ME OUT, WOMAN!
13. I'm not that miserable.
14. That bitch be cray
15. Inb4 faggots crying misogyny.
16. Ugly stupid bitch.
17. She's too mousy for me.
18. She's a stupid slut
19. I'd rape her
20. That bitch is garbage just like her old man
21. holy shit she is ugly
22. That's a huuuugggee bitch!
23. What a disgusting fat whore.
24. Truly an ugly woman
25. I'm the opposite. Peggy infuriates me for reasons I don't quite understand. I want to punch her in her smug face.
26. nah he a bitch
27. fuck her right in the ass!
28. Reddi

## Using more positive comments 

In [165]:
similar_comments_expanded = getKNNFromVector([vec3], embs, comments)
print_comments(similar_comments_expanded)

0. Ugly stupid bitch.
1. Oh shut up. You just hate women.
2. That bitch is ugly as fuck
3. can't stand FIFO's
4. Damn that's one ugly bitch.
5. Truly an ugly woman
6. take that back! [women are funny](http://i.imgur.com/UnJvgpX.gifv)
XD
7. What a dumb cunt.
8. I hate him
9. What a disgusting fat whore.
10. girls r so dumb
11. It must be that I'm a Dick because that bitch is not attractive
12. holy shit she is ugly
13. Fat bitches are gross.
14. That's a huuuugggee bitch!
15. Goddamn I hate kids.
16. Lol, stupid fucking chinklet.
17. She's a stupid slut
18. downright bitch imo
19. Stupid asshole
20. Fuck you, idiot.
21. I don't know your gender but stfu bitch.
22. Hahaha stupid women xD 😂😂😂
23. STUPID LITTLE FAGMAN FUCK YOU I HATE YOU BITCH
24. What a fucking asshole
25. That bitch be cray
26. Girl photogs... ugh.
27. Stupid fat worthless cunt.
28. I hate you.
29. Dumb cunt


In [166]:
print("***** AFTER INCORPORATING NEGATIVE COMMENTS INTO SIMILARITY ****** \n")
print_comment_diff(similar_comments_original, similar_comments_diff)

***** AFTER INCORPORATING NEGATIVE COMMENTS INTO SIMILARITY ****** 

COMMENTS THAT WERE REMOVED: 
0. I hate you.  Well-played.
1. Goddamn I hate kids.
2. I hate raisins
3. I dnt knw u.. But i hate you
4. Just awful
5. I don't like your kind.
6. I fuckin hate that kid
7. I just hate groups of people in general.
8. STUPID LITTLE FAGMAN FUCK YOU I HATE YOU BITCH
9. i dont give a shit
10. I don't like black people.
11. Houston.

Haters gonna hate.
12. Fuck off

https://www.reddit.com/r/pics/comments/5k33w2/optical_illusion/
https://www.reddit.com/r/funny/comments/5jfvbj/close_one_eye_tilt_your_phone_and_look_at_this/?ref=search_posts
https://www.reddit.com/r/funny/comments/5k2zlx/close_one_eye_tilt_your_phone_and_look_from_the/
https://www.reddit.com/r/pics/comments/5k3dec/optical_illusion/
https://www.reddit.com/r/funny/comments/5k3ney/look_at_it_from_the_charging_slot_of_your_phone/
https://www.reddit.com/r/pics/comments/5k3s9x/i_love_optical_illusions/
https://www.reddit.com/r/pics/comm

In [167]:
print("***** AFTER ADDING A FEW MORE POSITIVE EXAMPLES ****** \n")

print_comment_diff(similar_comments_original, similar_comments_expanded)

***** AFTER ADDING A FEW MORE POSITIVE EXAMPLES ****** 

COMMENTS THAT WERE REMOVED: 
0. I really hate women. Off topic but nonetheless true.
1. I hate you.  Well-played.
2. it was 'I hate everybody'

http://www.cracked.com/quick-fixes/7-hilarious-ways-badass-movie-lines-got-ruined-by-tv-censors/
3. I'm not that miserable.
4. I hate raisins
5. PLEASE KILL HER
6. I'd fuck her.
7. I dnt knw u.. But i hate you
8. No one can hate like a god lover
9. ... I'm a girl?
10. Just awful
11. I don't like your kind.
12. I fuckin hate that kid
13. I just hate groups of people in general.
14. i dont give a shit
15. I don't like black people.
16. Houston.

Haters gonna hate.
17. Fuck off

https://www.reddit.com/r/pics/comments/5k33w2/optical_illusion/
https://www.reddit.com/r/funny/comments/5jfvbj/close_one_eye_tilt_your_phone_and_look_at_this/?ref=search_posts
https://www.reddit.com/r/funny/comments/5k2zlx/close_one_eye_tilt_your_phone_and_look_from_the/
https://www.reddit.com/r/pics/comments/5k3dec/

In [170]:
print("***** COMPARING ADDING NEGATIVE VS MORE POSITIVE EXAMPLES ****** \n")

print_comment_diff(similar_comments_diff, similar_comments_expanded)


***** COMPARING ADDING NEGATIVE VS MORE POSITIVE EXAMPLES ****** 

COMMENTS THAT WERE REMOVED: 
0. I really hate women. Off topic but nonetheless true.
1. PLEASE KILL HER
2. I'd fuck her.
3. it was 'I hate everybody'

http://www.cracked.com/quick-fixes/7-hilarious-ways-badass-movie-lines-got-ruined-by-tv-censors/
4. ... I'm a girl?
5. I don't know her.
6. She is so hot to me.
7. LET ME OUT, WOMAN!
8. I'm not that miserable.
9. Inb4 faggots crying misogyny.
10. She's too mousy for me.
11. I'd rape her
12. That bitch is garbage just like her old man
13. I'm the opposite. Peggy infuriates me for reasons I don't quite understand. I want to punch her in her smug face.
14. nah he a bitch
15. fuck her right in the ass!
16. Reddit: "I don't like Amy Schumer because of her comedy; it's just not my style... has nothing to do with the fact she's a fat ugly cunt... I mean, she *is* a fat ugly cunt... but really, the only reason I dislike this fat ugly cunt is because her type of comedy isn't reall